In [1]:
import qctoolkit as qtk
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import clear_output
import re
from datetime import datetime

#%matplotlib inline

clear_output()

In [2]:
from horton import GOBasisFamily
#basis = GOBasisFamily('basis', filename='basis/sto2g/H_Ne_uncontracted.nwchem')
#basis = '3-21g'
#basis = 'sto-3g'
#basis = GOBasisFamily('H_He_basis', filename='basis/sto6g/H_Ne.nwchem')
basis = GOBasisFamily('H_He_basis', filename='basis/sto6g/H_Be-s.nwchem')

resolution = 0.001
batch_size = 2000

In [3]:
molstr_list = [
    'HH',
#     'HHe+',
#     'HLi',
#     'HBe+',
#     'HeHe',
#     'HeLi+',
#     'HeBe',
#     'LiLi',
#     'LiBe+',
#     'BeBe',
]

In [4]:
inp_grp = []
#inp_dump = []
for molstr in molstr_list:
    if '+' in molstr:
        charge = 1
    elif '-' in molstr:
        charge = -1
    else:
        charge = 0
    alist = re.findall('[A-Z][a-z]*', molstr)
    Zs = [qtk.n2Z(a) for a in alist]
    inp_list = []
    inp_grp.append(inp_list)
    #inp_dump.append(inp_list)
    res = resolution
    for R in np.arange(0.3, 4.0 + res, res):
        mol = qtk.Molecule()
        ZR = [[Zs[0], 0,0,0], [Zs[1], R, 0, 0]]
        mol.build(ZR)
        mol.charge = charge
        mol.name = mol.name + '_%s' % str(R)
        inp = qtk.QMInp(mol, program='horton', basis_set=basis, scf_step=30)
        try:
            inp.run()
            D, U = np.linalg.eigh(inp.olp)
            inp.X = U / np.sqrt(D)
            
            # to save inps into pickle file
            #inp.delete_ht_types()
            #inp.delete_matrices()
            
            inp_list.append(inp)
        except:
            pass
    print "%s finished" % mol.name
print [len(inp) for inp in inp_grp]

H2_4.001 finished
[3702]


In [5]:
# for i in range(len(inp_grp)):
#     inp = inp_grp[i]
#     E = np.array([o.Et for o in inp])
#     E = E-E[-1]
#     #print E
#     plt.plot(E, label=molstr_list[i])
# plt.ylim(-0.35, 0.001)
# plt.legend()

In [5]:
# Numpy version implementation

def get_input_tensor(mol):
    """Extract matrices and pack as tensor"""
    S, K, Na, C0, X, Er = mol.olp, mol.kin, mol.na, mol.initial_mov, mol.X, mol.er
    I = np.stack([S, K, Na, C0, X])
    nn = mol.ht_external['nn']
    occ = mol.ht_exp_alpha.occupations
    return I, Er, occ, nn
    
def get_output(I, Er, occ, nn, C):
    """take input tensor and approximated MO coefficients, C, to compute total energy"""
    K = I[1]
    Na = I[2]
    dm = (C * occ).dot(C.T)
    
    Ek = np.trace(dm.dot(K)) * 2
    Ev = np.trace(dm.dot(Na)) * 2
    Ej = np.trace(dm.dot(np.tensordot(dm, Er, axes=([0,1], [0,2])))) * 2
    Ex = -np.trace(dm.dot(np.tensordot(dm, Er, axes=([0,1], [0,1]))))
    E_tot = np.sum([Ek, Ev, Ej, Ex, nn])
    return E_tot

inp = inp_grp[0][5]
I_np, Er_np, occ, nn = get_input_tensor(inp)
C = inp.ht_exp_alpha.coeffs.__array__()
print get_output(I_np, Er_np, occ, nn, C)
print inp.Et

-0.648279412454
-0.648279412454


In [6]:
from sklearn.model_selection import train_test_split

def train_test_valid(valid_ind, train_inds=None):
    if valid_ind >= len(inp_grp):
        valid_ind = len(inp_grp) - 1
    print "%s is used as validation molecule" % molstr_list[valid_ind]
    inp_valid = inp_grp[valid_ind]
    if valid_ind > 0:
        if train_inds is None: train_inds = range(len(inp_grp))
        inps_all = list(qtk.flatten([inp_grp[i] for i in train_inds if i != valid_ind]))
    else:
        inps_all = list(qtk.flatten(inp_grp))
    inp_train, inp_test = train_test_split(inps_all, random_state=0, test_size=0.1)
    return inp_train, inp_test, inp_valid

def validation_inds(inp_valid):
    E = np.array([inp.Et for inp in inp_valid])
    ind_min = np.argmin(E)
    ind_max = len(E) - 1
    ind_mid = ind_min + (ind_max - ind_min) / 2
    ind_rep = ind_min / 2
    return ind_rep, ind_min, ind_mid, ind_max

In [8]:
def tfIO(mol_list, inds=None):
    I_list, Er_list, occ_list, nn_list, C_list, labels = [[] for _ in range(6)]
    
    if inds is None:
        inds = range(len(mol_list))
        
    for i in inds:
        mol = mol_list[i]
        C = mol.ht_exp_alpha.coeffs.__array__()
        C_list.append(C)
        I, Er, occ, nn = get_input_tensor(mol)
        I_list.append(I)
        Er_list.append(Er)
        occ_list.append(occ)
        nn_list.append(nn)
        labels.append(get_output(I, Er, occ, nn, C))
        
    return np.stack(I_list).astype(np.float64), np.stack(Er_list).astype(np.float64), \
           np.stack(occ_list).astype(np.float64), np.stack(nn_list).astype(np.float64), \
           np.stack(C_list).astype(np.float64), np.stack(labels).astype(np.float64)

def tfIO_batch(mol_list, batch_size, inds=None):
    
    def batches(mol_list):
        for i in range(0, len(mol_list), batch_size):
            yield mol_list[i:i + batch_size]

    I, Er, occ, nn, C, y = [], [], [], [], [], []
    if inds is None:
        for mol_batch in batches(mol_list):
            I_b, Er_b, occ_b, nn_b, C_b, y_b = tfIO(mol_batch)
            I.append(I_b)
            Er.append(Er_b)
            occ.append(occ_b)
            nn.append(nn_b)
            C.append(C_b)
            y.append(y_b)
    else:
        I_b, Er_b, occ_b, nn_b, C_b, y_b = tfIO(mol_list, inds)
        I.append(I_b)
        Er.append(Er_b)
        occ.append(occ_b)
        nn.append(nn_b)
        C.append(C_b)
        y.append(y_b)
    return I, Er, occ, nn, C, y

In [9]:
inp_train, inp_test, inp_valid = train_test_valid(0, [0])
# print len(inp_train), len(inp_test), len(inp_valid)
I_train, Er_train, occ_train, nn_train, C_train, y_train = tfIO_batch(inp_train, batch_size)
I_test, Er_test, occ_test, nn_test, C_test, y_test = tfIO_batch(inp_test, batch_size)
I_valid, Er_valid, occ_valid, nn_valid, C_valid, y_valid = tfIO_batch(inp_valid, batch_size)

I_valid2, Er_valid2, occ_valid2, nn_valid2, C_valid2, y_valid2 = tfIO(inp_valid, [1,3,5])
# print I_valid2.shape
# print I_test[0].shape
# print I_train[0].shape
# print validation_inds(inp_valid)

HH is used as validation molecule


In [17]:
import tensorflow as tf

def hidden_layer(I, C_prd = None, sess=None, keep_prob=0.8):
    
    if C_prd is None:
        #C_prd = I[-2]
        C_prd = I[:,-2]
        
    shape = C_prd.shape[-2:]
    S, K, V = I[:,0], I[:,1], I[:,2]
    
    Wc = tf.Variable(tf.truncated_normal(shape, dtype=tf.float64), name='weights_C', dtype=tf.float64)
    Bc = tf.Variable(tf.zeros(shape, dtype=tf.float64), name='biases_C', dtype=tf.float64)
    Ws = tf.Variable(tf.truncated_normal(shape, dtype=tf.float64), name='weights_S', dtype=tf.float64)
    Bs = tf.Variable(tf.zeros(shape, dtype=tf.float64), name='biases_S', dtype=tf.float64)
    Wk = tf.Variable(tf.truncated_normal(shape, dtype=tf.float64), name='weights_K', dtype=tf.float64)
    Bk = tf.Variable(tf.zeros(shape, dtype=tf.float64), name='biases_K', dtype=tf.float64)
    Wv = tf.Variable(tf.truncated_normal(shape, dtype=tf.float64), name='weights_V', dtype=tf.float64)
    Bv = tf.Variable(tf.zeros(shape, dtype=tf.float64), name='biases_V', dtype=tf.float64)
    
    if sess:
        sess.run(tf.global_variables_initializer())
    
    C_new = tf.add(tf.scan(lambda a, x: tf.matmul(x, Wc), C_prd), Bc)
    S_new = tf.add(tf.scan(lambda a, x: tf.matmul(x, Ws), S), Bs)
    K_new = tf.add(tf.scan(lambda a, x: tf.matmul(x, Wk), K), Bk)
    V_new = tf.add(tf.scan(lambda a, x: tf.matmul(x, Wv), V), Bv)
    
    new_matrix = tf.add(tf.add(tf.add(C_new, S_new), K_new), V_new)
    
    #return tf.nn.relu(new_matrix)
    return tf.nn.dropout(new_matrix, keep_prob)

def hamiltonian_layer(I, C_prd = None, sess = None):
    
    if C_prd is None:
        #C_prd = I[-2]
        C_prd = I[:,-2]
        
    shape = C_prd.shape[-2:]
    S, K, V = I[:,0], I[:,1], I[:,2]
    
    Wc = tf.Variable(tf.truncated_normal(shape, dtype=tf.float64), name='weights_C', dtype=tf.float64)
    Bc = tf.Variable(tf.zeros(shape, dtype=tf.float64), name='biases_C', dtype=tf.float64)
    
    if sess:
        sess.run(tf.global_variables_initializer())
    
    C_new = tf.add(tf.scan(lambda a, x: tf.matmul(x, Wc), C_prd), Bc)
    
    H_one = tf.add(tf.add(S, K), V)
    HC = tf.matmul(H_one, C_new)
    CT = tf.transpose(C_new, perm=[0,2,1])
    H_new = tf.matmul(CT, HC)
    
    _, C_prd = tf.self_adjoint_eig(H_new)
    
    return tf.matmul(I[:, -1], C_prd)

def normailization_layer(I, C_prd=None):
    
    if C_prd is None:
        #C_prd = I[-2]
        C_prd = I[:,-2]
        
    shape = C_prd.shape[-2:]
        
    W = tf.Variable(tf.truncated_normal(shape, dtype=tf.float64), name='weights', dtype=tf.float64)
    B = tf.Variable(tf.zeros(shape, dtype=tf.float64), name='biases', dtype=tf.float64)
    
    C_new = tf.add(tf.scan(lambda a, x: tf.matmul(x, W), C_prd), B)
    
    C_sym = tf.matmul(tf.transpose(C_new, perm=[0,2,1]), C_new)
    _, C_diag = tf.self_adjoint_eig(C_sym)
    
    return tf.matmul(I[:,-1], C_diag)

def output_layer(I, Er, occ, nn, C_prd, sess=None):
    K, Na = I[:,1], I[:,2]
    
    batch = tf.shape(I)[0]
    C_occ = tf.multiply(tf.expand_dims(occ, 1), C_prd)
    #C_occ = tf.matmul(tf.expand_dims(0, occ), C)
    dm = tf.matmul(C_occ, tf.transpose(C_prd, perm=[0,2,1]))
    
    ind = tf.transpose(tf.stack([tf.range(batch), tf.range(batch)]))
    J_kernel = tf.gather_nd(tf.tensordot(dm, Er, axes=([1,2], [1,3])), ind)
    X_kernel = tf.gather_nd(tf.tensordot(dm, Er, axes=([1,2], [1,2])), ind)
    
    Ek = tf.trace(tf.matmul(dm, K)) * 2
    Ev = tf.trace(tf.matmul(dm, Na)) * 2
    Ej = tf.trace(tf.matmul(dm, J_kernel)) * 2
    Ex = -tf.trace(tf.matmul(dm, X_kernel))
    
    E_total = tf.add(Ek, Ev)
    E_total = tf.add(E_total, Ej)
    E_total = tf.add(E_total, Ex)
    E_total = tf.add(E_total, nn)
    
    return E_total
    

# test
_N = 20
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    E = output_layer(I_test[0], Er_test[0], occ_test[0], nn_test[0], C_test[0], sess)
    C_orth = normailization_layer(I_test[0])
    print abs(sess.run(E) -y_test[0]).sum()

8.1379347705e-14


In [20]:
def train(root_name='model', learn_rate=0.001, valid_ind=2, 
          train_list=range(len(inp_grp)), print_step=5,
          threshold=1E-5, maxiter=100000, from_scratch=False
         ):
    
    def log_msg(msg, output=True):
        log.write(msg)
        log.flush()
        if output:
            print msg
    
    inp_train, inp_test, inp_valid = train_test_valid(valid_ind, train_list)
    print "train/test/validation size: %d/%d/%d" % (len(inp_train), len(inp_test), len(inp_valid))
    I_train, Er_train, occ_train, nn_train, C_train, y_train = tfIO_batch(inp_train, batch_size)
    I_test, Er_test, occ_test, nn_test, C_test, y_test = tfIO_batch(inp_test, batch_size)
    I_valid, Er_valid, occ_valid, nn_valid, C_valid, y_valid = tfIO_batch(inp_valid, batch_size)
    i_rep, i_min, i_mid, i_max = validation_inds(inp_valid)
    v_inds = [i_rep, i_min, i_mid, i_max]
    I_valid2, Er_valid2, occ_valid2, nn_valid2, C_valid2, y_valid2 = tfIO(inp_valid, inds=v_inds)
    
    status = "root name: " + root_name + "\n"
    status += "training set: " + str([molstr_list[i] for i in train_list]) + "\n"
    status += "validation: " + molstr_list[valid_ind] + "\n"
    status += "learning rate: " + str(learn_rate) + "\n"
    status += "print step: " + str(print_step) + "\n"
    status += "threshold: " + str(threshold) + "\n"
    status += "maxiter: " + str(maxiter) + "\n"
    status += "execuate time: " + datetime.now().strftime("%Y/%m/%d %H:%M:%S") + "\n\n"
    
    log = open("%s.log" % root_name, 'w')
    log_msg(status)

    tf.reset_default_graph()

    def get_shape(tensor):
        shape = list(tensor.shape)
        shape.insert(0, None)
        return shape

    # input tensors
    I = tf.placeholder(tf.float64, shape=get_shape(I_test[0][0]), name='I')
    Er = tf.placeholder(tf.float64, shape=get_shape(Er_test[0][0]), name='Er')
    occ = tf.placeholder(tf.float64, shape=get_shape(occ_test[0][0]), name='occ')
    nn = tf.placeholder(tf.float64, shape=get_shape(nn_test[0][0]), name='nn')

    # training tensors
    #C_ref = tf.placeholder(tf.float64, shape=get_shape(C_test[0][0]), name='C_ref')
    y = tf.placeholder(tf.float64, shape=get_shape(y_test[0][0]), name='y')

    # network architecture
    C_prd = hidden_layer(I)
#     C_prd = hamiltonian_layer(I, C_prd)
    for _ in range(10):
        #C_prd = hamiltonian_layer(I, C_prd)
        C_prd = hidden_layer(I, C_prd)
    #C_prd = hidden_layer(I, C_prd)
#     for _ in range(5):
#         #C_prd = hamiltonian_layer(I, C_prd)
#         C_prd = hidden_layer(I, C_prd)
    C_prd = hidden_layer(I, C_prd)
    #C_prd = hamiltonian_layer(I, C_prd)
    C_prd = normailization_layer(I, C_prd)
    C_prd = normailization_layer(I, C_prd)
    C_out = C_prd
    y_prd = output_layer(I, Er, occ, nn, C_prd)

    err = tf.pow(y - y_prd, 2)

    #optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(err)
    optimizer = tf.train.AdamOptimizer(learn_rate).minimize(err) # default learning rate 0.001

    err_list = []
    err_list_test = []
    itr_list = []
    E_prd_list = []
    C_prd_list = []

    # add C_prd_iteration history for post processing on the learning of the following properties
    # 1. density difference
    # 2. force
    # 3. dipole moment
    # 4. HOMO-LUMO gap

    saver = tf.train.Saver()
    save_path = "%s.ckpt" % root_name

    #with tf.Session() as sess:
    sess = tf.Session()
    try:
        train_err_hist = []
        test_err_hist = []
        valid_err_hist = []
        C_valid_hist = []

        if not from_scratch:
            try:
                saver.restore(sess, save_path)
                train_err_hist, test_err_hist, valid_err_hist, C_valid_hist = qtk.load('%s_hist.pkl' % root_name)
                msg = 'model and history loaded, continue optimizing'
            except:
                sess.run(tf.global_variables_initializer())
                msg = 'no model found, or reload failed, start from scratch'
        else:
            msg = 'start from scratch...'
            sess.run(tf.global_variables_initializer())
        log_msg(msg + "\n")
        test_err = 1
        epoch = 0 # i.e. epoch
        while test_err > threshold and epoch < maxiter:
        #for opt_itr in range(200):

            train_err_list = []
            for I_b, Er_b, occ_b, nn_b, C_b, y_b in zip(I_train, Er_train, occ_train, nn_train, C_train, y_train):
                train_dict = {I:I_b, Er:Er_b, occ:occ_b, nn:nn_b, y:y_b}
                sess.run(optimizer, feed_dict=train_dict)
                train_err_list.append(np.array(sess.run(err, feed_dict=train_dict)))
            train_err = np.concatenate(train_err_list).mean()
            train_err_hist.append(train_err)

            test_err_list = []
            for I_t, Er_t, occ_t, nn_t, C_t, y_t in zip(I_test, Er_test, occ_test, nn_test, C_test, y_test):
                test_dict = {I:I_t, Er:Er_t, occ:occ_t, nn:nn_t, y:y_t}
                test_err_list.append(np.array(sess.run(err, feed_dict=test_dict)))
            test_err = np.concatenate(test_err_list).mean()
            test_err_hist.append(test_err)

            valid_err_list = []
            for I_v, Er_v, occ_v, nn_v, C_v, y_v in zip(I_valid, Er_valid, occ_valid, nn_valid, C_valid, y_valid):
                valid_dict = {I:I_v, Er:Er_v, occ:occ_v, nn:nn_v, y:y_v}
                valid_err_list.append(np.array(sess.run(err, feed_dict=valid_dict)))
            valid_err = np.concatenate(valid_err_list).mean()
            valid_err_hist.append(valid_err)
            
            valid2_dict = {I:I_valid2, Er:Er_valid2, occ:occ_valid2, nn:nn_valid2, y:y_valid2}
            C_valid_hist.append(np.array(sess.run(C_out, feed_dict=valid2_dict)))

            msg = "epoch: %5d train/test/validatoin error: %f/%f/%f" % (epoch+1, train_err, test_err, valid_err)
            
            log_msg(msg + "\n", False)
            
            epoch += 1
            if epoch % print_step == 0:
                print msg
                qtk.save([train_err_hist, test_err_hist, valid_err_hist, C_valid_hist], "%s_hist.pkl" % root_name)
                
        save_path_out = saver.save(sess, save_path)
        msg = "Model saved in file: %s" % save_path_out
        log_msg(msg + "\n")
        qtk.save([train_err_hist, test_err_hist, valid_err_hist, C_valid_hist], "%s_hist.pkl" % root_name)
        print "optimization history saved"
        log.close()
        sess.close()
        print "session and log file closed"

    except KeyboardInterrupt:
        new_msg = "keyboard interrupt, current stage: %s" % msg
        log_msg(new_msg + "\n")
        save_path_out = saver.save(sess, save_path)
        msg = "Model saved in file: %s" % save_path_out
        log_msg(msg + "\n")
        qtk.save([train_err_hist, test_err_hist, valid_err_hist, C_valid_hist], "%s_hist.pkl" % root_name)
        print "optimization history saved"
        log.close()
        sess.close()
        print "session and log file closed"

In [21]:
train('H2_property', 0.02, 0)

HH is used as validation molecule
train/test/validation size: 3331/371/3702
root name: H2_property
training set: ['HH']
validation: HH
learning rate: 0.02
print step: 5
threshold: 1e-05
maxiter: 100000
execuate time: 2017/09/25 10:59:37


INFO:tensorflow:Restoring parameters from H2_property.ckpt
no model found, or reload failed, start from scratch

epoch:     5 train/test/validatoin error: 0.181105/0.161328/0.169367
epoch:    10 train/test/validatoin error: 0.127298/0.115752/0.125011
epoch:    15 train/test/validatoin error: 0.101059/0.092726/0.099850
epoch:    20 train/test/validatoin error: 0.076036/0.071864/0.073470
epoch:    25 train/test/validatoin error: 0.054922/0.059817/0.055828
epoch:    30 train/test/validatoin error: 0.042972/0.043612/0.042459
epoch:    35 train/test/validatoin error: 0.035668/0.034635/0.038149
epoch:    40 train/test/validatoin error: 0.032642/0.035544/0.033279
epoch:    45 train/test/validatoin error: 0.035089/0.040525/0.037185
epoch:    50 train/test/val

epoch:   575 train/test/validatoin error: 0.010781/0.012750/0.011078
epoch:   580 train/test/validatoin error: 0.010706/0.012239/0.011236
epoch:   585 train/test/validatoin error: 0.010747/0.012160/0.010794
epoch:   590 train/test/validatoin error: 0.010760/0.012328/0.011018
epoch:   595 train/test/validatoin error: 0.010633/0.012484/0.010838
epoch:   600 train/test/validatoin error: 0.010688/0.011924/0.011503
epoch:   605 train/test/validatoin error: 0.010755/0.017397/0.012497
epoch:   610 train/test/validatoin error: 0.010665/0.011875/0.010923
epoch:   615 train/test/validatoin error: 0.010672/0.011910/0.011105
epoch:   620 train/test/validatoin error: 0.010719/0.014098/0.011159
epoch:   625 train/test/validatoin error: 0.010728/0.015971/0.010720
epoch:   630 train/test/validatoin error: 0.010692/0.011968/0.012635
epoch:   635 train/test/validatoin error: 0.010649/0.012440/0.010719
epoch:   640 train/test/validatoin error: 0.010745/0.012101/0.011625
epoch:   645 train/test/validatoin

epoch:  1170 train/test/validatoin error: 0.006365/0.012720/0.006800
epoch:  1175 train/test/validatoin error: 0.006584/0.007733/0.007324
epoch:  1180 train/test/validatoin error: 0.006758/0.006718/0.008323
epoch:  1185 train/test/validatoin error: 0.006894/0.011929/0.007648
epoch:  1190 train/test/validatoin error: 0.006574/0.005615/0.006953
epoch:  1195 train/test/validatoin error: 0.008244/0.010275/0.008177
epoch:  1200 train/test/validatoin error: 0.007315/0.008583/0.007839
epoch:  1205 train/test/validatoin error: 0.007806/0.009541/0.007859
epoch:  1210 train/test/validatoin error: 0.006463/0.007391/0.008159
keyboard interrupt, current stage: epoch:  1212 train/test/validatoin error: 0.006992/0.008179/0.007337

Model saved in file: H2_property.ckpt

optimization history saved
session and log file closed


In [15]:
# def train(root_name='model', learn_rate=0.001, valid_ind=2, 
#           train_list=range(len(inp_grp)), print_step=5, from_scratch=False,
#           threshold=1E-5, maxiter=100000
#          ):

total_list = range(10)
tags = [str(i) for i in range(1, 10)]
inds = [total_list[:i] for i in range(1, 10)]

for i in range(len(tags)):
    tag = tags[i]
    ind = inds[i]
    train('increment_2-%s' % tag, 0.005, 2, ind, 10, 1E-5, 10)

HLi is used as validation molecule
train/test/validation size: 666/75/741
root name: increment_2-1
training set: ['HH']
validation: HLi
learning rate: 0.005
print step: 10
threshold: 1e-05
maxiter: 10
execuate time: 2017/09/23 23:45:25


INFO:tensorflow:Restoring parameters from increment_2-1.ckpt
no model found, or reload failed, start from scratch

epoch:    10 train/test/validatoin error: 1.081627/1.101953/10.626317
Model saved in file: increment_2-1.ckpt

optimization history saved
session and log file closed
HLi is used as validation molecule
train/test/validation size: 1333/149/741
root name: increment_2-2
training set: ['HH', 'HHe+']
validation: HLi
learning rate: 0.005
print step: 10
threshold: 1e-05
maxiter: 10
execuate time: 2017/09/23 23:45:43


INFO:tensorflow:Restoring parameters from increment_2-2.ckpt
no model found, or reload failed, start from scratch

epoch:    10 train/test/validatoin error: 2.922430/3.053366/13.224135
Model saved in file: increment_2-2.ckpt

optimiz